In [2]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(),"apperception"))

### IMPORTS
import cv2

from scenic_world import *
from world_executor import *
from video_util import *
from metadata_util import *
import json
import pandas as pd

from scenic_generate_df import *


In [3]:
### Let's define some attribute for constructing the world first
name = 'ScenicWorld' # world name
units = 'metrics'      # world units
user_data_dir = os.path.join("v1.0-mini")

In [5]:
# generate dataframes from scenic json files
# takes 1min to run
df_sample_data, df_annotation = scenic_generate_df()

In [6]:
scenic_world = ScenicWorld(name=name, units=units)

In [7]:
### Ingest the camera to the world
scene_name = "scene-0061"
scenic_world = scenic_world.scenic_camera(scene_name)

In [8]:
### Call execute on the world to run the detection algorithm and save the real data to the database
recognized_scenic_world = scenic_world.recognize(scene_name, df_sample_data, df_annotation)
recognized_scenic_world.execute()

Worlds Table created successfully........
New world inserted successfully........
Camera Table created successfully........
New camera inserted successfully.........
Recognization done, saving to database......


In [4]:
# the trajectory returned now includes the temporal info instead of just the spatio info
car_trajectories = scenic_world.predicate(lambda obj:obj.object_type == "vehicle.car").get_trajectory().execute()

Module(body=[Return(value=Compare(left=Attribute(value=Name(id='obj', ctx=Load()), attr='object_type', ctx=Load()), ops=[Eq()], comparators=[Str(s='vehicle.car')]))])
Worlds Table created successfully........
New world inserted successfully........
atPeriodSet(Test_Scenic_Item_General_Trajectory.trajCentroids,'{[0001-01-01 00:00:00, 9999-12-31 23:59:59.999999)}')
asMFJSON(atPeriodSet(Test_Scenic_Item_General_Trajectory.trajCentroids,'{[0001-01-01 00:00:00, 9999-12-31 23:59:59.999999)}'))::json
SELECT asMFJSON(atPeriodSet(Test_Scenic_Item_General_Trajectory.trajCentroids,'{[0001-01-01 00:00:00, 9999-12-31 23:59:59.999999)}'))::json From Test_Scenic_Item_General_Trajectory  Where Test_Scenic_Item_General_Trajectory.objectType='vehicle.car';



In [ ]:
print(car_trajectories[0][0])

In [8]:
bboxes = scenic_world.predicate(lambda obj:obj.object_type == "vehicle.car").get_geo().execute()

Module(body=[Return(value=Compare(left=Attribute(value=Name(id='obj', ctx=Load()), attr='object_type', ctx=Load()), ops=[Eq()], comparators=[Str(s='vehicle.car')]))])
Module(body=[Return(value=Compare(left=Call(func=Name(id='Tmin', ctx=Load()), args=[Attribute(value=Name(id='obj', ctx=Load()), attr='location', ctx=Load())], keywords=[]), ops=[GtE()], comparators=[Name(id='start', ctx=Load())]))])
Module(body=[Return(value=Compare(left=Call(func=Name(id='Tmax', ctx=Load()), args=[Attribute(value=Name(id='obj', ctx=Load()), attr='location', ctx=Load())], keywords=[]), ops=[Lt()], comparators=[Name(id='end', ctx=Load())]))])
Worlds Table created successfully........
New world inserted successfully........
Xmin(Test_Scenic_General_Bbox.trajBbox)
Ymin(Test_Scenic_General_Bbox.trajBbox)
Zmin(Test_Scenic_General_Bbox.trajBbox)
Xmax(Test_Scenic_General_Bbox.trajBbox)
Ymax(Test_Scenic_General_Bbox.trajBbox)
Zmax(Test_Scenic_General_Bbox.trajBbox)
SELECT Xmin(Test_Scenic_General_Bbox.trajBbox), 

In [9]:
bboxes.shape

(2004, 6)

In [13]:
import psycopg2
conn = psycopg2.connect(
        database="mobilitydb", user="docker", password="docker", host="localhost", port=25432)

In [14]:
from scenic_util import *
export_tables(conn)

NameError: name 'con' is not defined

In [15]:
cur = conn.cursor()
cur.execute(CREATE_CAMERA_SQL)
cur.execute(CREATE_BBOXES_SQL)
cur.execute(CREATE_ITEMTRAJ_SQL)
conn.commit()
with open('test_camera.csv', 'r') as camera_f:
    cur.copy_expert(file=camera_f, sql="COPY test_scenic_cameras FROM STDIN CSV HEADER DELIMITER as ','")
with open('test_trajectory.csv', 'r') as trajectory_f:
    cur.copy_expert(file=trajectory_f, sql="COPY test_scenic_item_general_trajectory FROM STDIN CSV HEADER DELIMITER as ','")
with open('test_bbox.csv', 'r') as bbox_f:
    cur.copy_expert(file=bbox_f, sql="COPY test_scenic_general_bbox FROM STDIN CSV HEADER DELIMITER as ','")

conn.commit()